In [1]:
import numpy as np

from a_nice_mc.models.discriminator import MLPDiscriminator
from a_nice_mc.models.generator import create_nice_network
from a_nice_mc.train.wgan_nll import Trainer

from hepmc.core.densities.qcd import ee_qq_ng
from hepmc.core.phase_space.mapping import MappedDensity
from hepmc.core.phase_space.rambo import RamboOnDiet

In [2]:
import tensorflow as tf
from a_nice_mc.objectives import Energy
from tensorflow.python.framework import ops

class QCDTarget(Energy):
    def __init__(self):
        super(QCDTarget, self).__init__()
        self.name = "QCDTarget"
        self.z = tf.placeholder(tf.float32, [None, 8], name='z')
        target = ee_qq_ng(2, 100., 5., .3)
        rambo_mapping = RamboOnDiet(100., 4)
        self.mapped = MappedDensity(target, rambo_mapping)
    
    def __call__(self, z):
        return self.tf_energy(z)
    
    # energy as numpy function
    def energy(self, z):
        return self.mapped.pot(z).astype(np.float32, copy=False)
        #return self.camel.pot(z).view('float32')
    
    # energy as tensorflow function
    def tf_energy(self, z, name=None):
        with tf.name_scope(name, "energy", [z]) as name:
            y = tf.py_func(self.energy,
                       [z],
                       [tf.float32],
                       name=name,
                       stateful=False)
            return y[0]
    
    #@staticmethod
    #def mean():
    #    return np.array(3*[.5])
    
    #@staticmethod
    #def std():
    #    return np.array(3*[.181])
    
    def evaluate(self, zv, path=None):
        pass

In [3]:
energy_fn = QCDTarget()

In [4]:
#def noise_sampler(bs):
#    return np.random.normal(0.0, 1.0, [bs, 8])

def noise_sampler(bs):
    return np.random.uniform(0.0, 1.0, [bs, 8])

#def noise_sampler(bs):
#    noise = np.empty((bs, 8))
#    
#    indices = np.arange(bs)
#    while indices.size > 0:
#        x = np.random.normal(.5, .25, [indices.size, 8])
#        y = energy_fn.mapped.pot(x)
#        in_bounds = np.logical_not(np.isinf(y))
#        noise[indices[in_bounds]] = x[in_bounds]
#        indices = indices[np.logical_not(in_bounds)]
#
#    return noise

In [5]:
discriminator = MLPDiscriminator([400, 400, 400])
generator = create_nice_network(
    8, 10,
    [
        ([400], 'v1', False),
        ([400, 400], 'x1', True),
        ([400], 'v2', False)
    ]
)

In [6]:
trainer = Trainer(generator, energy_fn, discriminator, noise_sampler, b=16, m=4)

In [7]:
trainer.train(max_iters=10000)

before bootstrap
before hmc_sampler.sample
Event 1	(batch acceptance rate: 0.000000)
Event 2	(batch acceptance rate: 0.000000)	max sequence length: 3
Event 3	(batch acceptance rate: 0.000000)	max sequence length: 4
Event 4	(batch acceptance rate: 0.000000)	max sequence length: 5
Event 5	(batch acceptance rate: 0.000000)	max sequence length: 6
Event 6	(batch acceptance rate: 0.000000)	max sequence length: 7
Event 7	(batch acceptance rate: 0.142857)	max sequence length: 7
Event 8	(batch acceptance rate: 0.125000)	max sequence length: 7
Event 9	(batch acceptance rate: 0.222222)	max sequence length: 7
Event 10	(batch acceptance rate: 0.300000)	max sequence length: 7
Event 20	(batch acceptance rate: 0.250000)	max sequence length: 7
Event 30	(batch acceptance rate: 0.200000)	max sequence length: 12
Event 40	(batch acceptance rate: 0.175000)	max sequence length: 12
Event 50	(batch acceptance rate: 0.160000)	max sequence length: 12
Event 60	(batch acceptance rate: 0.100000)	max sequence length

/net/theorie/home/tjansse2/studium/masterarbeit/hep-monte-carlo/src/hepmc/core/markov/metropolis.py:45: RuntimeWarning: invalid value encountered in subtract
  return state.pot - candidate.pot


Event 200	(batch acceptance rate: 0.000000)	max sequence length: 85
Event 300	(batch acceptance rate: 0.160000)	max sequence length: 103
Event 400	(batch acceptance rate: 0.240000)	max sequence length: 103
Event 500	(batch acceptance rate: 0.040000)	max sequence length: 103
Event 600	(batch acceptance rate: 0.020000)	max sequence length: 116
Event 700	(batch acceptance rate: 0.080000)	max sequence length: 116
Event 800	(batch acceptance rate: 0.140000)	max sequence length: 116
Event 900	(batch acceptance rate: 0.220000)	max sequence length: 116
Event 1	(batch acceptance rate: 0.000000)
Event 2	(batch acceptance rate: 0.000000)	max sequence length: 3
Event 3	(batch acceptance rate: 0.000000)	max sequence length: 4
Event 4	(batch acceptance rate: 0.000000)	max sequence length: 5
Event 5	(batch acceptance rate: 0.000000)	max sequence length: 6
Event 6	(batch acceptance rate: 0.000000)	max sequence length: 7
Event 7	(batch acceptance rate: 0.000000)	max sequence length: 8
Event 8	(batch ac

[a_nice_mc.train.wgan_nll] [INFO] Iter [0] time [0.0000] d_loss [7.2935] g_loss [0.3763] v_loss [0.4949]
[a_nice_mc.train.wgan_nll] [INFO] Iter [100] time [12.6747] d_loss [-0.5023] g_loss [1.2331] v_loss [0.5150]
[a_nice_mc.train.wgan_nll] [INFO] Iter [200] time [24.5971] d_loss [-0.4098] g_loss [1.1282] v_loss [0.5082]
[a_nice_mc.train.wgan_nll] [INFO] Iter [300] time [35.9730] d_loss [-0.3423] g_loss [1.1312] v_loss [0.4965]
[a_nice_mc.train.wgan_nll] [INFO] Iter [400] time [47.2993] d_loss [-0.1941] g_loss [1.0680] v_loss [0.5121]
before bootstrap
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [740.0130] samples/s [259.4549]
before sample
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [746.7715] samples/s [257.1068]
z: [[[0.1310895  0.72236794 0.78580064 ... 0.26246718 0.6450074  0.14586364]
  [0.1310895  0.72236794 0.78580064 ... 0.26246718 0.6450074  0.14586364]
  [0.1310895  0.72236794 0.78580064 ... 0.26246718 0.

z: [[[0.19199684 0.450985   0.77741086 ... 0.8859615  0.68276256 0.5611403 ]
  [0.19199684 0.450985   0.77741086 ... 0.8859615  0.68276256 0.5611403 ]
  [0.19199684 0.450985   0.77741086 ... 0.8859615  0.68276256 0.5611403 ]
  ...
  [0.578299   0.21087821 0.6996738  ... 0.27097788 0.68539435 0.7234413 ]
  [0.578299   0.21087821 0.6996738  ... 0.27097788 0.68539435 0.7234413 ]
  [0.578299   0.21087821 0.6996738  ... 0.27097788 0.68539435 0.7234413 ]]

 [[0.37987942 0.56834614 0.9818369  ... 0.90038896 0.36129192 0.7597252 ]
  [0.37987942 0.56834614 0.9818369  ... 0.90038896 0.36129192 0.7597252 ]
  [0.37987942 0.56834614 0.9818369  ... 0.90038896 0.36129192 0.7597252 ]
  ...
  [0.07507819 0.83574384 0.2758174  ... 0.41790375 0.07077986 0.41538766]
  [0.07507819 0.83574384 0.2758174  ... 0.41790375 0.07077986 0.41538766]
  [0.07507819 0.83574384 0.2758174  ... 0.41790375 0.07077986 0.41538766]]

 [[0.41769513 0.93981475 0.06121557 ... 0.7154783  0.85672337 0.9425001 ]
  [0.41769513 0.939

...

[a_nice_mc.train.wgan_nll] [INFO] Iter [5600] time [716.7421] d_loss [-0.0434] g_loss [0.1966] v_loss [0.4830]
[a_nice_mc.train.wgan_nll] [INFO] Iter [5700] time [734.8714] d_loss [-0.1867] g_loss [0.3316] v_loss [0.5387]
[a_nice_mc.train.wgan_nll] [INFO] Iter [5800] time [753.2136] d_loss [-0.1768] g_loss [0.6691] v_loss [0.5538]
[a_nice_mc.train.wgan_nll] [INFO] Iter [5900] time [771.6355] d_loss [-0.0399] g_loss [0.2801] v_loss [0.4832]
before bootstrap
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [838.9867] samples/s [228.8475]
before sample
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [794.5173] samples/s [241.6562]
z: [[[0.15367559 0.81679827 0.0998749  ... 0.526073   0.25862753 0.01414261]
  [0.15367559 0.81679827 0.0998749  ... 0.526073   0.25862753 0.01414261]
  [0.15367559 0.81679827 0.0998749  ... 0.526073   0.25862753 0.01414261]
  ...
  [0.49106452 0.2911678  0.67812335 ... 0.6607161  0.9143036  0.099984

[a_nice_mc.train.wgan_nll] [INFO] Iter [6600] time [863.5972] d_loss [-0.0191] g_loss [0.0347] v_loss [0.4782]
[a_nice_mc.train.wgan_nll] [INFO] Iter [6700] time [876.8039] d_loss [-0.1305] g_loss [0.4934] v_loss [0.5263]
[a_nice_mc.train.wgan_nll] [INFO] Iter [6800] time [888.2052] d_loss [-0.1083] g_loss [0.2167] v_loss [0.5328]
[a_nice_mc.train.wgan_nll] [INFO] Iter [6900] time [899.8443] d_loss [-0.0967] g_loss [0.3025] v_loss [0.5317]
before bootstrap
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [814.9045] samples/s [235.6104]
before sample
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [835.4401] samples/s [229.8190]
z: [[[0.36057395 0.37184858 0.35630637 ... 0.544893   0.9484012  0.2684823 ]
  [0.26967117 0.51435715 0.97309047 ... 0.12565562 0.44260532 0.17082246]
  [0.09050333 0.49197444 0.726907   ... 0.5923939  0.662415   0.55652916]
  ...
  [0.06509508 0.5899679  0.0761582  ... 0.9436443  0.8409084  0.997499

[a_nice_mc.train.wgan_nll] [INFO] Iter [7600] time [995.9269] d_loss [-0.0886] g_loss [0.0781] v_loss [0.5459]
[a_nice_mc.train.wgan_nll] [INFO] Iter [7700] time [1006.8072] d_loss [-0.1212] g_loss [0.2521] v_loss [0.5579]
[a_nice_mc.train.wgan_nll] [INFO] Iter [7800] time [1017.5600] d_loss [-0.0505] g_loss [0.0255] v_loss [0.5090]
[a_nice_mc.train.wgan_nll] [INFO] Iter [7900] time [1028.3551] d_loss [-0.1239] g_loss [0.1754] v_loss [0.5196]
before bootstrap
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [794.7427] samples/s [241.5876]
before sample
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [820.5481] samples/s [233.9899]
z: [[[0.5917412  0.26465496 0.02979662 ... 0.973854   0.45956233 0.93050456]
  [0.5917412  0.26465496 0.02979662 ... 0.973854   0.45956233 0.93050456]
  [0.5917412  0.26465496 0.02979662 ... 0.973854   0.45956233 0.93050456]
  ...
  [0.21058822 0.5831614  0.9761059  ... 0.85826284 0.85192883 0.474

[a_nice_mc.train.wgan_nll] [INFO] Iter [8600] time [1112.8251] d_loss [-0.0974] g_loss [0.1999] v_loss [0.4995]
[a_nice_mc.train.wgan_nll] [INFO] Iter [8700] time [1124.2055] d_loss [-0.1246] g_loss [0.4068] v_loss [0.5532]
[a_nice_mc.train.wgan_nll] [INFO] Iter [8800] time [1135.2384] d_loss [-0.1268] g_loss [0.4132] v_loss [0.5214]
[a_nice_mc.train.wgan_nll] [INFO] Iter [8900] time [1146.7000] d_loss [-0.0605] g_loss [0.4641] v_loss [0.5160]
before bootstrap
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [780.2883] samples/s [246.0629]
before sample
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [792.1693] samples/s [242.3724]
z: [[[0.27629715 0.47078153 0.9757344  ... 0.9484759  0.01247916 0.9639127 ]
  [0.27629715 0.47078153 0.9757344  ... 0.9484759  0.01247916 0.9639127 ]
  [0.27629715 0.47078153 0.9757344  ... 0.9484759  0.01247916 0.9639127 ]
  ...
  [0.6175031  0.53425086 0.8330947  ... 0.59171236 0.45778364 0.15

[a_nice_mc.train.wgan_nll] [INFO] Iter [9600] time [1236.9768] d_loss [-0.0750] g_loss [0.3490] v_loss [0.5181]
[a_nice_mc.train.wgan_nll] [INFO] Iter [9700] time [1247.7895] d_loss [-0.1698] g_loss [0.3412] v_loss [0.5884]
[a_nice_mc.train.wgan_nll] [INFO] Iter [9800] time [1258.1667] d_loss [-0.1097] g_loss [0.2613] v_loss [0.5787]
[a_nice_mc.train.wgan_nll] [INFO] Iter [9900] time [1268.8730] d_loss [-0.0418] g_loss [0.3159] v_loss [0.5266]


In [8]:
sample, _ = trainer.sample(batch_size=32, steps=2000)

[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [2000 : 1] time [248.0711] samples/s [257.9906]


In [9]:
burn_in = 1000
sample = sample[:, burn_in:]

In [10]:
sample = np.reshape(sample, (-1, 8))

In [11]:
n_accepted = 1
for i in range(1, 32000):
    if (sample[i] != sample[i-1]).any():
        n_accepted += 1
print(n_accepted)

1788


In [12]:
from hepmc.core.sampling import Sample
from hepmc.core.densities.qcd import export_hepmc
rambo_mapping = RamboOnDiet(100., 4)
anicemc_sample = Sample(data=rambo_mapping.map(sample), weights=np.full(sample.shape[0], 1./sample.shape[0]))
export_hepmc(100., anicemc_sample, "../samples/qcd/2-4/a_nice_mc.hepmc")

In [1]:
sample_container = Sample(data=sample[0][0])

NameError: name 'Sample' is not defined

In [ ]:
mean = sample_container.mean
variance = sample_container.variance

In [ ]:
from hepmc.core.util.stat_tools import effective_sample_size

In [ ]:
effective_sample_size(sample_container, mean, variance)